In [25]:
import os
import pandas as pd
fpRaw = "./Environment_Temperature_change_E_All_Data_NOFLAG.csv"
fpOut = "./DeltaTempByYear.csv"

In [33]:
#https://www.kaggle.com/datasets/sevgisarac/temperature-change/
dfRaw = pd.read_csv(fpRaw,encoding='cp1252')
print(dfRaw.shape)
print(dfRaw.columns)

(9656, 66)
Index(['Area Code', 'Area', 'Months Code', 'Months', 'Element Code', 'Element',
       'Unit', 'Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967',
       'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 'Y1974', 'Y1975',
       'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983',
       'Y1984', 'Y1985', 'Y1986', 'Y1987', 'Y1988', 'Y1989', 'Y1990', 'Y1991',
       'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999',
       'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007',
       'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Y2014', 'Y2015',
       'Y2016', 'Y2017', 'Y2018', 'Y2019'],
      dtype='object')


In [34]:
# Data contains many kinds of observations
# Keep only change in Temperature data by year
df = dfRaw
df = df[df.Months == "Meteorological year" ]
df = df[df.Element == "Temperature change"]
# All datum with area code above 5000 are aggregates e.g. AFRICA
df = df[df["Area Code"] < 5000]
# Many small island nations have very long, conglomirated names
# Remove the longer names for sake of presentation
lenTable = list(len(area) <30 for area in df["Area"])
df = df[lenTable]
print(df.shape)


(235, 66)


In [35]:
# Aggregate data by decade to reduce volitility and reduce storage space
# Remove incomplete entries and keep only the new columns and state name
def getColNames(decade):
    years = range(0,10)
    return ["Y"+ str(decade[:-2])+str(year) for year in years]

decades = ["1960s","1970s","1980s","1990s","2000s","2010s"]
df["Y1960"] = 0
for decade in decades:
    years = getColNames(decade)
    df[decade] = df[years].mean(axis=1)
    #.mean(skipna=True,numeric_only=True, axis=0)
df = df[["Area"]+decades]
df = df.dropna()
print(df.shape)
df.head()

(192, 7)


,Area,1960s,1970s,1980s,1990s,2000s,2010s
32,Afghanistan,-0.1294,0.0317,0.2145,0.344700,0.970400,1.1188
66,Albania,0.1005,-0.1760,0.0346,0.396700,0.927000,1.5816
100,Algeria,0.0972,-0.1315,0.5516,0.799800,1.338700,1.5400
134,American Samoa,-0.1020,0.0145,0.3040,0.714667,0.782889,1.1611
168,Andorra,0.0773,-0.1164,0.4487,0.961500,1.199000,1.5096


In [14]:
#Write Background Data to File
fileBG = "dataBG.json"
fpBG = os.path.join(os.getcwd(),"wllm","data", fileBG)         
file = open(fpBG, "w")
toFile = df.to_json(orient="split")
file.write(toFile)
file.close()